In [92]:
import os
os.chdir("/mnt/c/cloud/thesis/")
import math

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd

import tokenizers

import seaborn as sns
sns.set(rc={'figure.figsize':(15, 10)})

from einops import rearrange

from constants import paths as p
from constants import tokens as t
from constants import hyperparameters as hp

import pytorch_lightning as pl
import pytorch_lightning.callbacks as cb
from models.transformer import Transformer

In [4]:
t = nn.Transformer()

In [7]:
loss = nn.CrossEntropyLoss()

In [78]:
loss2 = nn.CrossEntropyLoss(ignore_index=0)

In [9]:
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))

In [15]:
tgt_mask = torch.zeros((32, 20))

In [16]:
tgt_mask[:, 18:] = 1

In [18]:
out = t.forward(src, tgt, tgt_key_padding_mask=tgt_mask)

In [19]:
out.shape

torch.Size([20, 32, 512])

In [ ]:
batch_size x sequence_length(padded with a certain index) x vocabulary_length

In [ ]:
docs say: batch_size x vocabulary_length

In [62]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

In [86]:
input = torch.randn(32, 256, 100).transpose(1, 2)
target = torch.empty(32, 256, dtype=torch.long).random_(100)
target[:, 50:] = 0

In [89]:
input.shape, target.shape

(torch.Size([32, 100, 256]), torch.Size([32, 256]))

In [90]:
loss(input, target)

tensor(5.1161)

In [91]:
loss2(input, target)

tensor(5.1420)

torch.Size([3, 5])

In [25]:
loss2.forward(input, target)

tensor(0.8711, grad_fn=<NllLossBackward>)

In [2]:
class IMDB_Reviews(pl.LightningDataModule):
    def __init__(self, train_path=p.TRAIN_TENSOR_DATASET_PATH, validaiton_path=p.VALIDATION_TENSOR_DATASET_PATH, batch_size=hp.BATCH_SIZE):
        super().__init__()
        self.train_path = train_path
        self.validation_path = validaiton_path
        self.batch_size = batch_size

    def setup(self, stage):
        self.train_data = torch.load(self.train_path)
        self.validation_data = torch.load(self.validation_path)

    def train_dataloader(self):
        return DataLoader(self.train_data, self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.validation_data, self.batch_size)

In [3]:
class TransformerLightning(pl.LightningModule):
    def __init__(self, vocabulary_size=hp.VOCABULARY_SIZE, embedding_size=hp.EMBEDDING_SIZE, number_of_properties=hp.NUMBER_OF_PROPERTIES, padding_index=hp.PADDING_INDEX, model_dimension=hp.MODEL_DIMENSION, target_sequence_length=hp.TARGET_SEQUENCE_LENGTH, dropout_probability=hp.DROPOUT_PROBABILITY, feed_forward_transformer_layer_dimension=hp.FEED_FORWARD_TRANSFORMER_LAYER_DIMENSION):
        super().__init__()
        self.transformer = Transformer(vocabulary_size, embedding_size, number_of_properties, padding_index, model_dimension, target_sequence_length, dropout_probability, feed_forward_transformer_layer_dimension)
        self.loss_function = nn.CrossEntropyLoss(ignore_index=t.PADDING_TOKEN)
        
        self.train_name = "train"
        self.validation_name = "validation"
        
    def forward(self, batched_source_numericalized, batched_source_properties, batched_source_padding_mask):
        pass
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=6*10e-5)
    
    def training_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, self.train_name)

    def validation_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, self.validation_name)

    def _step(self, batch, batch_idx, mode):
        bsn, bsp, btin, bspm, btipm, bton = batch
        logits = self.transformer(bsn, bsp, btin, bspm, btipm)
        
        # logits -> batch_size x target_sequence_length x vocabulary_size
        # bton -> batch_size x target_sequence_length([0, vocabulary_size-1])
        loss = self.loss_function(rearrange(logits, 'b t v -> b v t'), bton)

        on_step = True if mode == self.train_name else False
        self.log(f'{mode}_loss', loss, on_step=on_step, on_epoch=True, prog_bar=True, logger=True)

        return loss
        
       

SyntaxError: invalid syntax (<ipython-input-3-a0c833a48a3c>, line 7)